In [459]:
import numpy as np
import sklearn 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

#To delete SettingWithCopyWarning (huge gain of time)
pd.options.mode.chained_assignment = None  # default='warn'

In [460]:
df=pd.read_csv("data/train.csv")
test=pd.read_csv("data/test.csv")

In [461]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [462]:
#Preprocessing
#train[['Cabin','Embarked']].describe()
df.describe(include="all")

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Cohen, Mr. Gurshon ""Gus""",male,NaN,NaN,NaN,CA. 2343,NaN,G6,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


In [463]:
#Preprocessing
#CLEANING
df_clean=df
df_clean=df_clean.dropna(subset = ['Pclass', 'Age','Fare','Sex','Embarked','SibSp','Parch'])
#print(np.isreal(df_clean[['Fare']]).sum())

In [464]:
#SUBSTITUTING: Sex=1 eq "male" && Sex=0 eq "female"  
for i in range(np.shape(df_clean)[0]):
    if df_clean.Sex.iloc[i]=='male':
        df_clean.Sex.iloc[i]=1
    else:
        df_clean.Sex.iloc[i]=0
df_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [465]:
#SUBSTITUTING: Embarked=0 eq "S" && EMbarked=1 eq "C" && Embarked=2 eq "Q"  
for i in range(np.shape(df_clean)[0]):
    if df_clean.Embarked.iloc[i]=='S':
        df_clean.Embarked.iloc[i]=0
    elif df_clean.Embarked.iloc[i]=='C':
        df_clean.Embarked.iloc[i]=1
    else:
        df_clean.Embarked.iloc[i]=2
df_clean.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0


In [466]:
df_clean.Embarked.unique()

array([0, 1, 2], dtype=int64)

In [502]:
#Split dataset set into training and testing sets
train,test=train_test_split(df_clean,test_size=0.2)
print("np.shape(train) =",np.shape(train))
print("np.shape(test) =",np.shape(test))
X=train[['Pclass','Age','Sex','Embarked','SibSp','Parch']]
y=train['Survived']
print("np.shape(X) =",np.shape(X))
print("np.shape(y) =",np.shape(y))

np.shape(train) = (569, 12)
np.shape(test) = (143, 12)
np.shape(X) = (569, 6)
np.shape(y) = (569,)


In [503]:
#NORMALIZE DATA (not standardize since we don't know if data follow a Gaussian or not: 
#https://machinelearningmastery.com/normalize-standardize-machine-learning-data-weka/)
def function_normalize(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df

In [504]:
#Learning Algorithm
clf=svm.SVC()
X=function_normalize(X)
clf.fit(X,y)
X.head()

,0,1,2,3,4,5
0,1.0,0.522493,1.0,0.0,0.0,0.0
1,0.0,0.635587,1.0,0.5,0.0,0.2
2,1.0,0.447097,0.0,0.0,0.2,0.0
3,0.5,0.421965,1.0,0.0,0.0,0.0
4,1.0,0.208344,1.0,0.5,0.2,0.2


In [505]:
#Testing
X_test=test[['Pclass','Age','Sex','Embarked','SibSp','Parch']]
X_test=function_normalize(X_test)
y_pred=clf.predict(X_test)

In [506]:
accuracy_score(test[['Survived']], y_pred)

0.8181818181818182

In [507]:
scores = cross_val_score(clf, test[['Survived']], y_pred, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.82 (+/- 0.23)


In [508]:
#Multi Layer Perceptron (NN)
clfNN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clfNN.fit(X,y)
y_pred_NN=clfNN.predict(X_test)

In [509]:
accuracy_score(test[['Survived']], y_pred_NN)

0.8391608391608392

In [538]:
#Run in once
#Split dataset set into training and testing sets
train,test=train_test_split(df_clean,test_size=0.2)
print("np.shape(train) =",np.shape(train))
print("np.shape(test) =",np.shape(test))
X=train[['Pclass','Age','Sex','Embarked','SibSp','Parch']]
y=train['Survived']
print("np.shape(X) =",np.shape(X))
print("np.shape(y) =",np.shape(y))
#NORMALIZE DATA (not standardize since we don't know if data follow a Gaussian or not: https://machinelearningmastery.com/normalize-standardize-machine-learning-data-weka/)
def function_normalize(df):
    x = df.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    return df
#Learning Algorithm
clf=svm.SVC()
X=function_normalize(X)
clf.fit(X,y)
X.head()
#Testing
X_test=test[['Pclass','Age','Sex','Embarked','SibSp','Parch']]
X_test=function_normalize(X_test)
y_pred=clf.predict(X_test)
print("Accuracy Score SVM:",accuracy_score(test[['Survived']], y_pred))
#Multi Layer Perceptron (NN)
clfNN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clfNN.fit(X,y)
y_pred_NN=clfNN.predict(X_test)
print("Accuracy score NN: ",accuracy_score(test[['Survived']], y_pred_NN))

np.shape(train) = (569, 12)
np.shape(test) = (143, 12)
np.shape(X) = (569, 6)
np.shape(y) = (569,)
Accuracy Score SVM: 0.7902097902097902
Accuracy score NN:  0.8461538461538461


In [ ]:
y_pred_NN=clfNN.predict(X_test)